In [2]:
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import nn, optim

data_dir = "C:/Users/paulj/Downloads/ML_project/CUB_200_2011/CUB_200_2011/images"

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(128),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

subset_classes = ["001.Black_footed_Albatross", "002.Laysan_Albatross"]
datasets_dict = {
    x: datasets.ImageFolder(
        os.path.join(data_dir),
        transform=data_transforms[x]
    ) for x in ['train', 'val']
}

dataloaders = {
    x: DataLoader(datasets_dict[x], batch_size=64, shuffle=True, num_workers=2)
    for x in ['train', 'val']
}

dataset_sizes = {x: len(datasets_dict[x]) for x in ['train', 'val']}
class_names = datasets_dict['train'].classes

model = models.mobilenet_v2(pretrained=True)
num_ftrs = model.last_channel
model.classifier[1] = nn.Linear(num_ftrs, len(class_names))
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, criterion, optimizer, dataloaders, num_epochs=5):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

model = train_model(model, criterion, optimizer, dataloaders, num_epochs=5)

torch.save(model.state_dict(), 'bird_classifier_fast.pth')

print("Modèle entraîné et sauvegardé avec succès !")


c:\Users\paulj\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\paulj/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 62.2MB/s]


Epoch 1/5
----------
train Loss: 3.7392 Acc: 0.1625
val Loss: 2.5877 Acc: 0.3248
Epoch 2/5
----------
train Loss: 2.6969 Acc: 0.3207
val Loss: 1.8606 Acc: 0.4840
Epoch 3/5
----------
train Loss: 2.2920 Acc: 0.4076
val Loss: 1.5446 Acc: 0.5601
Epoch 4/5
----------
train Loss: 2.0831 Acc: 0.4554
val Loss: 1.3701 Acc: 0.6050
Epoch 5/5
----------
train Loss: 1.8933 Acc: 0.5003
val Loss: 1.2467 Acc: 0.6361
Modèle entraîné et sauvegardé avec succès !
